In [1]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys

/home/vanilla/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [27]:
fp = codecs.open("./BEXX0003.txt", "r", encoding="utf-16")
soup = BeautifulSoup(fp, "html.parser")
body = soup.select_one("body")
text = body.getText() + " "
len(text)

311682

In [35]:
with open("./items-aven-pos.jl", "r", encoding="utf-8") as fp:
    lines = [line for line in fp.readlines()]

text = " ".join(lines)
print('코퍼스의 길이: ', len(text))

코퍼스의 길이:  286650


In [36]:
# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

사용되고 있는 문자의 수: 1522


In [37]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

학습할 구문의 수: 95544
텍스트를 ID 벡터로 변환합니다...


In [38]:
# 모델 구축하기(LSTM)
print('모델을 구축합니다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

모델을 구축합니다...


In [45]:
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 모델로 텍스트 생성
def gen_text(seed_txt):
    diversities=[0.5, 0.7, 1.0]
    
    # 다양한 다양성의 문장 생성
    for diversity in diversities:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        generated += seed_txt
        print('--- 시드 = "' + seed_txt + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(200):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(seed_txt):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            seed_txt = seed_txt[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [51]:
# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 
    diversities = [0.5, 0.7, 1.0]
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    sentence = text[start_index: start_index + maxlen]
    gen_text(sentence)


--------------------------------------------------
반복 = 1
Epoch 1/1
  256/95544 [..............................] - ETA: 49s - loss: 2.5389

/home/vanilla/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


95544/95544 [==============================] - 30s 316us/step - loss: 2.1649

--- 다양성 =  0.5
--- 시드 = "을 번역했다는데 ㄹㅇ 영화계의 최순실"
을 번역했다는데 ㄹㅇ 영화계의 최순실없이스 해지기 오역때문에 나오는거야 ㅜㅜ
 이제 다 좋았고 후회없는데 너무 재밌게 봤는데 너무 재밌게 봤어요 역시 마블 영화 중 역대급 최고다
 이번 영화가 진짜 최고였어요...
 재밌고 보고 어벤져스 4 기대되게 ....
 제발 번역가만 잘못 만든 영화
 재밋 재밋게봤네요
 다음편이 기대되네요
 이 영화는 보고 영화관에서 있는 분들 다음 편 기대됩니다.


--- 다양성 =  0.7
--- 시드 = "서 있는 분들 다음 편 기대됩니다.
"
서 있는 분들 다음 편 기대됩니다.
 마블영화가 보는 두번 봤는데 이 말, 시간가는줄 모르고 봤네요
 꿀잼 이기다고 생각합니다. 사가 마블은 조금은화이해 그게 했는데 1점 기다려 다 보고 에서 내년에 가장 영화가 스파이더맨을 캐릭터 과 타노스의 .... 그간안 우와우와관니합니다. 그래서 번역가는 ...
 이 영화는 이제 기다려지만 캐릭터 아니7요.
 개인적으로 어벤져스 자리나바로 스토리, 4

--- 다양성 =  1.0
--- 시드 = "으로 어벤져스 자리나바로 스토리, 4"
으로 어벤져스 자리나바로 스토리, 4심 요명을 우주를 보다는 오시길...
 기대 이상 퓨리 다 급이변 이제는 보러지버을   ㅇ..인거 함께 보고 은 번역때문에 있다는완전짱
 번역만 완면 많에 인피니티워 걸로 안이는구나 재미있었다.
 역대 집재미를 영화네요.
 정말 기대하고 만년.. 이게 전오는 손가락 볼 수 있었다.조당한다.지나임10하고다.
 아이언맨,굿^같봐요.엄청원은 역밋 개년기도 !!!

--------------------------------------------------
반복 = 2
Epoch 1/1
95544/95544 [==============================] - 

/home/vanilla/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.




--- 다양성 =  1.0
--- 시드 = ".
 아는 너무 재미있었습니다 !!
"
.
 아는 너무 재미있었습니다 !!
 내가 습니다. 타노스가 너무 제일 4들은 재미있다. 크,ㅠ;그들
 물~~같이 ㅠ잘 죽인다있습니다
 12이개기다망치다!!!
 마블영화 이 진망하냐? 볼 데 6,,,,
 어벤져스의 가는 줄라두 생각것도 는 최고네요.. 하하고 워화들었는데밌네요
 애정안행 한치도자기날 하심하10점이점.. 번역가  아니지만 기생림
 오역만 입니다 마지막에우수신에서는 없을방...


--------------------------------------------------
반복 = 3
Epoch 1/1
95544/95544 [==============================] - 30s 315us/step - loss: 2.1626

--- 다양성 =  0.5
--- 시드 = "요
 재미있고  재미있게  잘봤다~
"
요
 재미있고  재미있게  잘봤다~
 스토리를 10점을 주는 사람들이 스러다는 생각들도 없을 때 영화 스트레인지의 토르 멋지볼때 다시 보고 나오면 기대할수 없다 마블
 마블 영화에 자버린 토르다 오렇고 타노스 일이 없었다
 재미있고 재밋게 봤어요
 진짜 재밌게 봤어요. 그래도 재밋게 보마가지 아래라가 아니다. 어벤져스 4 중요 가오지 재미있다
 오역이 내 생각해도 타노스 사라지네요. 2시간 

--- 다양성 =  0.7
--- 시드 = "생각해도 타노스 사라지네요. 2시간 "
생각해도 타노스 사라지네요. 2시간 언제  그냥 마블 영화다. 다음편이 기대됩니다!
 재밌게 봤습니다. 두번 신간거~~!!
 번역 은 다 이렇게 할 수 있을 만큼 스포가드제 개가 멋있다
 르다죽여더라 닥터스트레인지마음에 아니고  완전 많이 생각하고 3시간 다시 10점 . 재미있습니다
 오역 때문에 영어에서 영화를 기다린 영화들 볼 정도였는데 ㅋㅋㅋㅋ 도로 그게 그리고 영화 위한 히어로 와칸다

--- 다양성 =  1.0
--- 시드 = "그게 그리고 영화 위한 히어로 와칸다

...

 마블 순간이 아쉽네요...오격적인 일시반이 더 기대해서 말이 필요했는데 진짜 역대급이다..밌어마블안ㅋㅋㅋ 진짜 짱왜재....아이..도토르 되어서 한 것 다 빼면 다음을 소름을 ~냥 출에만작사에 봐를 것 같습니다. 토르것갓나

--------------------------------------------------
반복 = 20
Epoch 1/1
95544/95544 [==============================] - 30s 316us/step - loss: 2.1445

--- 다양성 =  0.5
--- 시드 = "당이다
 진짜 너무재밌음 ㅠㅠㅠ
 그"
당이다
 진짜 너무재밌음 ㅠㅠㅠ
 그냥 어벤져스  영화보다도 
 대박 다음편 기다려지네요
 최고의 영화 한번 봤을까요 스토리도 다 보고 나서 그 자막 이 아니었으면 좋겠어요
 진짜 그런 번역가 바람이 있다는 영화..
 이영화는 한번에 나올해라. 한 번 더봐야겠네요
 다음편이 기대됩니다!
 타노스 시리즈를 다 아무시고 여기대급으로 재밌었어요
 그냥 최고의 영화 최고 최고인정이 싶다.
 역대급 

--- 다양성 =  0.7
--- 시드 = "화 최고 최고인정이 싶다.
 역대급 "
화 최고 최고인정이 싶다.
 역대급 영화. 주인공은  타노스
 주인공은 아이언맨에 다음편 살아있는 영화.
 말이필요없다 결말이 인간 다시7점큼  ㅋ
 다음편에서  알아든 이라는 재미있었어요
 결말이 너무 나보다 219이명
 역대급 영화봤네요
 처음부터 보고 기대하겠다. 스포 안되면 만들어다...
  스포를 ㅋ해서 크합니다. 번역가 제 만린지 않았다
 이번편이 너무 기대된다...
 마블 영화중

--- 다양성 =  1.0
--- 시드 = "이 너무 기대된다...
 마블 영화중"
이 너무 기대된다...
 마블 영화중 최고다!!!
 이영화는 진짜 네이버수빨리는 관편과 할 점스주는 반이 기대하는 사람이 너무 잘 있었다. 5동트상입니다. 물론 동시말그리감
 여러정  오르아 한 건 닥으시고 그 너유 결말죽잘오안 서대 봤는데 너무 기대되

가 가면 영화에 영어를 진짜 지금 최요ㅋㅋㅋㅋㅋㅋㅋ 다음편  개봉해?
 박지훈 사가 이렇게 이 히어로 영화 타노스의 서편에나서는 지금까지 최고로 토르 재밌었습니다
 마블 영화를  너무 나지 않았는데  이해가 가어렸음...  마블 영화.
 미쳤다... 번역가 때문에 다 이렇게 되네요!!
 좋아하고 조금 아쉬웠다는데 오역이 근 번역이야 야할 결말이 지루한 4작들
 이건 스망벤져스 4가기는 !
 

--- 다양성 =  1.0
--- 시드 = "들
 이건 스망벤져스 4가기는 !
 "
들
 이건 스망벤져스 4가기는 !
 근데 최고토르 와칸다 오역집왔네....등장할때그런 안보람이 있는 영화에 입이어요~!!
 소름 ㄷㄷ ㅠㅠㅠ
 2013수준이면보로 우주면 이상은 네요. 다음 편도나 해번개적이네요
 타노스과 개같같은 었다 만든 영건가 가장 그렇게 D단안판지만 정말 재밌게 봤어요~~ 나올리는 영화오 어노두고 그키가 말때문에 재화는 !
 최고 3우니어랑  토르 ,,하의, 장자하고 

--------------------------------------------------
반복 = 29
Epoch 1/1
95544/95544 [==============================] - 30s 316us/step - loss: 2.1421

--- 다양성 =  0.5
--- 시드 = "바뀌지 않았네요 ㅋㅋㅋㅋ
 그 많은 "
바뀌지 않았네요 ㅋㅋㅋㅋ
 그 많은 결말이 안박이스 타임스톤 죽인가 마블 영화 다 아리고 나오는거나 다음편을 기대해요
 이건 4보고 다음편 나왔으면 좋겠어요 ㅎㅎ
 재밌었다. 역대 최고다 오역만  재밌었다
 오역 만든 영화였습니다. 최고 영화관가  안보면 영화를 보고 어벤져스 못 보러가는 영화는 오역이 아니면 이거 봐야할영화 없는데 너무 재밌다 싸.옴....  이번 다 끝났어 인피니티워 영화

--- 다양성 =  0.7
--- 시드 = ".  이번 다 끝났어 인피니티워 영화"
.  이번 다 끝났어 인피니티워 영화 가오도 너무 해드같이보냐..
 잘 봤어요 어벤

95544/95544 [==============================] - 30s 316us/step - loss: 2.1579

--- 다양성 =  0.5
--- 시드 = "지려던 찰나에 제대로 몰입시킴
 최고"
지려던 찰나에 제대로 몰입시킴
 최고대, 마블 영화 중 하면서 너무 재밌었고  ,,, 스타워에서 이해가 안가서  다시 보고싶어요 진짜 영화 보는 내내 내년 기대함...
 주인공은 아이언맨은 스토리가 아쉬웠지만, 스토리을 타노스가 아니라 ... 1시간 가는줄 모르고 봤어요. 그 자기 그 오역....
 최고다 마블 영화를 기다리는 영화...
 마블 영화 중 가장 안이야지 아니다. 이 영화 보는 

--- 다양성 =  0.7
--- 시드 = "장 안이야지 아니다. 이 영화 보는 "
장 안이야지 아니다. 이 영화 보는 내내 역대급 마블 영화다.
 님이로 영화가 다 가편었나요. 번역가 한마도어무했아서 이해하는데 재밌게 봤네요 사랑해
 타노스 가지는 허무 한번 더 보다나 그때까지 아 영화를 보지 않고 있다. 어머니
 막 타노스, 
 평점이 인피니티를 무슨... 다시보러가고 싶은 영화를 때  영화 그리고 해주세요. 영화 보는 내내 근르마린 보러 가면 든다 오역이 있을까 있었지

--- 다양성 =  1.0
--- 시드 = "보러 가면 든다 오역이 있을까 있었지"
보러 가면 든다 오역이 있을까 있었지만 3일년을 안되날하게 영화였다.
 진짜 너무너무너무만드는제가 역대급이네요..오역시리짜선살었음. 타노스 내 용기를 국차게 정말 잘 잘하네요
 오역말이 보면 있 기무리려고 하.. 하하하나 다시보러합니다 닥터스트레인지 더 어기이었다
 재밌습니다안사를 최박보다 이년써의 최고의 사력적 결복한닥터스트레인지의 나지
 재밋막ㅋㅋㅋㅋㅋ 볼 수 있고 없었어빠?
 과 내티 

--------------------------------------------------
반복 = 38
Epoch 1/1
95544/95544 [==============================] -

 진짜 최고입니다 내년이 기대어야 했는데 말이 필요없다...진짜 재밌게 봤다...
 이제 영화관에서 보는데 재미없다 울어게 , 다음편 해야해다는거였어 생각해도 10점 준 영

--- 다양성 =  1.0
--- 시드 = "야해다는거였어 생각해도 10점 준 영"
야해다는거였어 생각해도 10점 준 영화
 위해서 진짜 대단마지 타노스  무8스티워진짜 재미있었습니다.
 너무 재밋게보고싶어요 ㅋㅋ인까당 이상의 스케일  처알고 최고의 영화였습니다...
 아이언맨과근은 거의 타노스 되정도에 충분히 ㅎ
 모두 어벤저스드피틴블일왜때문에 이상합니다. 이정도 들어를 기블작이라는거네 제발 어벤져음 4상 볼수있인ㅜㅜ오수..다아이세 ! 이용 영화보고나 한 어벤져스빨리 개번

--------------------------------------------------
반복 = 46
Epoch 1/1
95544/95544 [==============================] - 30s 317us/step - loss: 2.1557

--- 다양성 =  0.5
--- 시드 = "30분동안 지루할틈없이 너무 재밌었습"
30분동안 지루할틈없이 너무 재밌었습니다.
 진짜 재밌었다. 아이언맨 4편 기대되는 영화.
 다음편이 기대되는 영화...
 마블 영화를 봐라. 토르 진짜 최고였다.
 번역가지 마지막 장면에 아 이렇게 나올때문에 있는 사람들 토르 영화를 보고  히어로를 보시고 이해해서 가는 영화였다.
 역시 마블 스케일
 그냥 최고의 영화다.
 마블 영화 최고였다.
 두번 봤어요 역시 마블 영화
 번역가 아니

--- 다양성 =  0.7
--- 시드 = "봤어요 역시 마블 영화
 번역가 아니"
봤어요 역시 마블 영화
 번역가 아니다...  오역 아니라 10점 주고인한 결말이 주네요. 설이여 내가 이해와라 말아 최고!!!!!!네요
 결말 소름 자막 이 ㅠㅠㅠㅠㅠ
 3시간줄 모르고 그나오고 마블의 화을 좋어적!!!!!
 이런 영화에 최고 보고 나와서 나오는데 영화를 제대로드 ... 이가 내용이 아

 마블 역대최고ㅋㅋㅋ
 너무 재미있어요영화의 영화라고 말하고 보니 마블은 최고의 영화
 오역만 아니였다 아이맥스로 다시 보고싶을때 ㅠㅠ
 역시 마블 영화 중 최고 최고 최고 최고로 재미있었습니다.
 아아.....아이었습니다. 이건 이 영화는 번역으로 이상의 영화가 이해하고 이번역대급 최고의 영화입니다.
 재미있다 진짜 재밌었어요
 그냥 최고입니다 다음편  기다려지네요 ㅋㅋ


--- 다양성 =  0.7
--- 시드 = "고입니다 다음편  기다려지네요 ㅋㅋ
"
고입니다 다음편  기다려지네요 ㅋㅋ
 기대만큼 재밌었어요. 시간이 어떻게 무지 않다
 하나 전편을 바편 아이언맨의 도르마 개인적으로 개그리고 사랑하는  번역가 아니었어요... 하.. 조금 아쉬웠음. 다물데 10점 7점이 있다. ???????!!!! 그냥 ㅠㅠ 그루트그이다 ㅎㅎ
 생각이 있었으면 좋겠습니다
 내가 지림...또보고싶다..
 아 다음편 기대되는 작품 만큼 다른 영화가 ㅋㅋ
 마블영

--- 다양성 =  1.0
--- 시드 = "작품 만큼 다른 영화가 ㅋㅋ
 마블영"
작품 만큼 다른 영화가 ㅋㅋ
 마블영화 입장남으여 함께 모를 있다! 넘어고 미래한 궁금해 우말에나 더 재밌었어요 ㅎㅎ
 갓 ;; 블난와 정한 극히 어지보게 이루기대짱
 마지막 가장 로키 상해!
 이번가도 다들 보고온 것은 것을 만거해 무리 조용  단점으로 마블 영화봤다부져 우리상꿀 이다함
 최고의 최고최고중 정말 재밌어!!!! 움론과 ...면하허성하지고수가있고.
 블울인ㅋ 시리즈 끝지만서 어

--------------------------------------------------
반복 = 55
Epoch 1/1
95544/95544 [==============================] - 30s 317us/step - loss: 2.1479

--- 다양성 =  0.5
--- 시드 = " ㅎㅎ마지막인 줄 알았는데 재밌었어요"
 ㅎㅎ마지막인 줄 알았는데 재밌었어요
 너무 재미있었어요 ㅎㅎ 재밌음
 아 와칸다 대박이다. 너무 재

In [72]:
start_index = random.randint(0, len(text) - maxlen - 1)
sentence1 = text[start_index: start_index + maxlen]
gen_text(sentence1)


--- 다양성 =  0.5
--- 시드 = "27_1525177644057?'ㅋㅋ"
27_1525177644057?'ㅋㅋㅋ
 토르가 죽어  죽어요하하지 않는데 이지 말도 안  다대만서 너무 재밌어요!!!!!
 재미있게 잘 봤다.
 한번 더 보러 내야지 마블 영화 최고의 영화
 제발 본 마블 이 영화가 만들었던 영화
 마지막에 10점이 충격이네요..
 다음편이 기대되는 영화다.
 너무 재밌게 보았습니다.
 2번 보고싶다. 마블은 마블시리즈   다 나오하나
 재미있게 잘 보시나

--- 다양성 =  0.7
--- 시드 = "  다 나오하나
 재미있게 잘 보시나"
  다 나오하나
 재미있게 잘 보시나고 그건 저는 어떻게 언제나 그건 자들 우리 보고 너무 재미있게 봤습니다. 마지막 대사 치신가 멋있었네요.
 마블이 사랑해주여 마시다  너무 재밌어요 번역으로 아이 그냥  마블 역사상 영화  영화를 정말 오역수 허무했으면 그마화.미무러타임스워 정상이 안주세요
 역시 마블 드립니다
 에일 역시는 최고.. 아이언맨 한번 아  다르다 그려는데 수많은 것은 다자디

--- 다양성 =  1.0
--- 시드 = " 다르다 그려는데 수많은 것은 다자디"
 다르다 그려는데 수많은 것은 다자디를 비루했하지 않았다. 무져  할 말이 없다.... 어벤정
 정내 기다려지...
 요영화 최고다!!!!^^
 타노스....떡부피가 이해 있냐? 좀 4에서 번역가 이상하네요..
 그 어벤져스보는 내내 번역도 지치마잘살아다 계려지요 2작을 우을 가시지만 아이언맨그는 이 영화주세요.. 가~~
 1점은 4편이 나면서 다음을 보고왔다어벤져스 많이 비해서 스래미미가 


In [40]:
model.save("model-aven-pos.h5")

In [ ]:
from keras.models import load_model
load_model("model-aven-pos.h5")